# 📚 מדריך מקיף: עבודה עם מבני נתונים מקוננים ב-Polars

## תוכן עניינים

1. [מבוא ודרישות טכניות](#מבוא)
2. [יצירת רשימות (Lists)](#יצירת-רשימות)
3. [צבירת אלמנטים ברשימות](#צבירת-אלמנטים)
4. [גישה ובחירת אלמנטים](#גישה-ובחירה)
5. [החלת לוגיקה על אלמנטים](#החלת-לוגיקה)
6. [עבודה עם Structs ו-JSON](#עבודה-עם-structs)
7. [תרגילים ואתגרים](#תרגילים)
8. [סיכום ומשאבים נוספים](#סיכום)

---

<a id="מבוא"></a>
## 🎯 מבוא

### מהם מבני נתונים מקוננים?

מבני נתונים מקוננים הם מבנים שמכילים בתוכם מבנים אחרים. בPolars, אנחנו עובדים בעיקר עם:

- **רשימות (Lists)**: עמודות שכל תא בהן מכיל רשימה של ערכים
- **מבנים (Structs)**: עמודות שכל תא בהן מכיל מספר שדות (כמו dictionary)
- **JSON**: נתונים היררכיים שניתן לפרק ולעבד

### למה זה חשוב?

✅ **ביצועים גבוהים**: Polars מטפל ביעילות במבנים מורכבים  
✅ **גמישות**: ניתן לאחסן מידע מורכב בעמודה אחת  
✅ **שאילתות מתקדמות**: פעולות חזקות על נתונים היררכיים  

### דרישות טכניות

In [1]:
# התקנת Polars (אם עדיין לא מותקן)
# !pip install polars

# ייבוא הספרייה
import polars as pl
import warnings
warnings.filterwarnings('ignore')

# הצגת גרסה
print(f"🐻 Polars Version: {pl.__version__}")

🐻 Polars Version: 1.34.0


### טעינת הנתונים

נעבוד עם מערך נתונים של סרטוני YouTube שהיו פופולריים (trending) בארה"ב.

In [2]:
# טעינת הנתונים
# שים לב: יש להתאים את הנתיב לקובץ ה-CSV שלך
df = pl.read_csv("../data/us_videos.csv", try_parse_dates=True)

# המרת עמודת התאריכים לפורמט נכון
df = df.with_columns(
    pl.col("trending_date").str.to_date(format="%y.%d.%m")
)

# הצגת מידע כללי
print(f"📊 מספר שורות: {df.height:,}")
print(f"📊 מספר עמודות: {df.width}")
print(f"\n🔍 5 השורות הראשונות:")
df.head()

📊 מספר שורות: 40,949
📊 מספר עמודות: 16

🔍 5 השורות הראשונות:


video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
str,date,str,str,i64,"datetime[μs, UTC]",str,i64,i64,i64,i64,str,bool,bool,bool,str
"""2kyS6SvSYSE""",2017-11-14,"""WE WANT TO TALK ABOUT OUR MARR…","""CaseyNeistat""",22,2017-11-13 17:13:01 UTC,"""SHANtell martin""",748374,57527,2966,15954,"""https://i.ytimg.com/vi/2kyS6Sv…",false,false,false,"""SHANTELL'S CHANNEL - https://w…"
"""1ZAPwfrtAFY""",2017-11-14,"""The Trump Presidency: Last Wee…","""LastWeekTonight""",24,2017-11-13 07:30:00 UTC,"""last week tonight trump presid…",2418783,97185,6146,12703,"""https://i.ytimg.com/vi/1ZAPwfr…",false,false,false,"""One year after the presidentia…"
"""5qpjK5DgCt4""",2017-11-14,"""Racist Superman | Rudy Mancuso…","""Rudy Mancuso""",23,2017-11-12 19:05:24 UTC,"""racist superman|rudy|mancuso|k…",3191434,146033,5339,8181,"""https://i.ytimg.com/vi/5qpjK5D…",false,false,false,"""WATCH MY PREVIOUS VIDEO ▶ \n\n…"
"""puqaWrEC7tY""",2017-11-14,"""Nickelback Lyrics: Real or Fak…","""Good Mythical Morning""",24,2017-11-13 11:00:04 UTC,"""rhett and link|gmm|good mythic…",343168,10172,666,2146,"""https://i.ytimg.com/vi/puqaWrE…",false,false,false,"""Today we find out if Link is a…"
"""d380meD0W0M""",2017-11-14,"""I Dare You: GOING BALD!?""","""nigahiga""",24,2017-11-12 18:01:41 UTC,"""ryan|higa|higatv|nigahiga|i da…",2095731,132235,1989,17518,"""https://i.ytimg.com/vi/d380meD…",false,false,false,"""I know it's been a while since…"


In [3]:
# בואו נראה את מבנה הנתונים
df.glimpse(max_items_per_column=2)

Rows: 40949
Columns: 16
$ video_id                             <str> '2kyS6SvSYSE', '1ZAPwfrtAFY'
$ trending_date                       <date> 2017-11-14, 2017-11-14
$ title                                <str> 'WE WANT TO TALK ABOUT OUR MARRIAGE', 'The Trump Presidency: Last Week Tonight with John Oliver (HBO)'
$ channel_title                        <str> 'CaseyNeistat', 'LastWeekTonight'
$ category_id                          <i64> 22, 24
$ publish_time           <datetime[μs, UTC]> 2017-11-13 17:13:01+00:00, 2017-11-13 07:30:00+00:00
$ tags                                 <str> 'SHANtell martin', 'last week tonight trump presidency|last week tonight donald trump|john oliver trump|donald trump'
$ views                                <i64> 748374, 2418783
$ likes                                <i64> 57527, 97185
$ dislikes                             <i64> 2966, 6146
$ comment_count                        <i64> 15954, 12703
$ thumbnail_link                       <str> 'https://i.ytimg

### 💡 **טיפ חשוב**

> לפני שמתחילים לעבוד עם נתונים מקוננים, חשוב להבין את מבנה הנתונים המקורי. השתמשו ב-`glimpse()` ו-`describe()` כדי לחקור את הנתונים!

---
<a id="יצירת-רשימות"></a>
## 📝 1. יצירת רשימות (Creating Lists)

### מהי רשימה ב-Polars?

רשימה (List) היא עמודה שבה כל תא מכיל **וקטור של ערכים**. זה שונה מעמודה רגילה שבה כל תא מכיל **ערך בודד**.

### שלוש דרכים ליצור רשימות:

1. **פיצול מחרוזות** - `str.split()`
2. **צבירת ערכים** - `group_by()` + `agg()`
3. **איחוד עמודות** - `concat_list()`

### 🔹 דרך 1: פיצול מחרוזות (String Split)

בעמודת `tags` יש תגיות מופרדות בתו `|`. בואו נפצל אותן לרשימה:

In [4]:
# פיצול עמודת התגיות
result = df.select(
    'tags',
    pl.col('tags').str.split('|').alias('tags_list')
).head()

print("🔍 התוצאה:")
result

🔍 התוצאה:


tags,tags_list
str,list[str]
"""SHANtell martin""","[""SHANtell martin""]"
"""last week tonight trump presid…","[""last week tonight trump presidency"", ""last week tonight donald trump"", … ""donald trump""]"
"""racist superman|rudy|mancuso|k…","[""racist superman"", ""rudy"", … "" Lele Pons""]"
"""rhett and link|gmm|good mythic…","[""rhett and link"", ""gmm"", … ""challenge""]"
"""ryan|higa|higatv|nigahiga|i da…","[""ryan"", ""higa"", … ""fail""]"


In [5]:
# בואו נראה את הטיפוס של העמודה החדשה
print("📌 טיפוס העמודה המקורית:", df.select('tags').dtypes[0])
print("📌 טיפוס העמודה החדשה:", result.select('tags_list').dtypes[0])

📌 טיפוס העמודה המקורית: String
📌 טיפוס העמודה החדשה: List(String)


### 💡 **הסבר**

- `str.split('|')` מפצל מחרוזת לפי התו `|`
- התוצאה היא עמודה מטיפוס `list[str]` (רשימה של מחרוזות)
- כל תא מכיל כעת **רשימה** במקום מחרוזת בודדת

### 🔹 דרך 2: צבירת ערכים (Aggregation)

כאשר אנחנו מבצעים `group_by()`, אפשר לצבור ערכים לתוך רשימות:

In [6]:
# צבירת כל מזהי הסרטונים לפי תאריך
videos_by_date = (
    df
    .group_by('trending_date')
    .agg(pl.col('video_id'))
    .sort('trending_date', descending=True)
).head()

print("📅 סרטונים לפי תאריך:")
videos_by_date

📅 סרטונים לפי תאריך:


trending_date,video_id
date,list[str]
2018-06-14,"[""-QPdRfqTnt4"", ""gPHVLxm8U-0"", … ""ooyjaVdt-jA""]"
2018-06-13,"[""FchkqXEg0qs"", ""uHRwMmwbFnA"", … ""Q5KmA3Xbmqo""]"
2018-06-12,"[""PPWDwBrUNyY"", ""rAH8qm5oQHg"", … ""6S9c5nnDd_s""]"
2018-06-11,"[""0bXCbVGb04A"", ""L4pkD78oKSo"", … ""r-3iathMo7o""]"
2018-06-10,"[""L4pkD78oKSo"", ""ZFwylDNpgFc"", … ""r-3iathMo7o""]"


In [ ]:
# כמה סרטונים יש בכל תאריך?
videos_by_date.with_columns(
    pl.col('video_id').list.len().alias('num_videos')
)

### 💡 **הסבר**

- `group_by('trending_date')` מקבץ שורות לפי תאריך
- `agg(pl.col('video_id'))` אוסף את כל מזהי הסרטונים **לרשימה אחת**
- כל תאריך מקבל רשימה של כל הסרטונים שהיו טרנדיים באותו היום

### 🔹 דרך 3: איחוד עמודות (Concatenating Columns)

נוכל לקחת מספר עמודות ולאחד אותן לרשימה אחת:

In [9]:
# איחוד מדדי מעורבות לרשימה אחת
engagement_list = df.select(
    pl.concat_list(
        pl.col('views'),
        pl.col('likes'),
        pl.col('dislikes'),
        pl.col('comment_count')
    ).alias('engagement_metrics')
)

print("📊 מדדי מעורבות:")
print(engagement_list)

📊 מדדי מעורבות:
shape: (40_949, 1)
┌──────────────────────────────┐
│ engagement_metrics           │
│ ---                          │
│ list[i64]                    │
╞══════════════════════════════╡
│ [748374, 57527, … 15954]     │
│ [2418783, 97185, … 12703]    │
│ [3191434, 146033, … 8181]    │
│ [343168, 10172, … 2146]      │
│ [2095731, 132235, … 17518]   │
│ …                            │
│ [1685609, 38160, … 2657]     │
│ [1064798, 60008, … 3936]     │
│ [1066451, 48068, … 3992]     │
│ [5660813, 192957, … 13088]   │
│ [10306119, 357079, … 144795] │
└──────────────────────────────┘


In [10]:
# בואו נראה איך זה נראה עם שמות העמודות
df.select(
    'title',
    'views',
    'likes', 
    'dislikes',
    'comment_count',
    pl.concat_list('views', 'likes', 'dislikes', 'comment_count').alias('all_metrics')
).head(3)

title,views,likes,dislikes,comment_count,all_metrics
str,i64,i64,i64,i64,list[i64]
"""WE WANT TO TALK ABOUT OUR MARR…",748374,57527,2966,15954,"[748374, 57527, … 15954]"
"""The Trump Presidency: Last Wee…",2418783,97185,6146,12703,"[2418783, 97185, … 12703]"
"""Racist Superman | Rudy Mancuso…",3191434,146033,5339,8181,"[3191434, 146033, … 8181]"


### 💡 **הסבר**

- `concat_list()` מאחד ערכים מעמודות שונות **לרשימה בתא אחד**
- השורה הראשונה: `[748374, 57527, 2966, 15954]`
  - 748,374 צפיות
  - 57,527 לייקים
  - 2,966 דיסלייקים
  - 15,954 תגובות

### 🎓 תרגיל 1: יצירת רשימות

**משימה**: צור רשימה שמכילה את שם הערוץ, הכותרת ומספר הצפיות לכל סרטון.

In [11]:
# כתוב את הקוד שלך כאן:
# רמז: השתמש ב-concat_list עם העמודות הנכונות
df.select(
    'title',
    'views',
    'channel_title', 
    pl.concat_list('title', 'channel_title', 'views').alias('all_metrics')
).head(3)


title,views,channel_title,all_metrics
str,i64,str,list[str]
"""WE WANT TO TALK ABOUT OUR MARR…",748374,"""CaseyNeistat""","[""WE WANT TO TALK ABOUT OUR MARRIAGE"", ""CaseyNeistat"", ""748374""]"
"""The Trump Presidency: Last Wee…",2418783,"""LastWeekTonight""","[""The Trump Presidency: Last Week Tonight with John Oliver (HBO)"", ""LastWeekTonight"", ""2418783""]"
"""Racist Superman | Rudy Mancuso…",3191434,"""Rudy Mancuso""","[""Racist Superman | Rudy Mancuso, King Bach & Lele Pons"", ""Rudy Mancuso"", ""3191434""]"


### 📌 רשימות מקוננות (Nested Lists)

Polars תומך גם ברשימות של רשימות!

In [12]:
# יצירת רשימה מקוננת
nested_df = pl.DataFrame({
    'nested_list': [
        [[1,2,3], [4,5,6], [7,8,9], [10,11,12]], 
        [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]
    ]
})

print("🔢 רשימה מקוננת:")
print(f"טיפוס: {nested_df.dtypes[0]}")
nested_df

🔢 רשימה מקוננת:
טיפוס: List(List(Int64))


nested_list
list[list[i64]]
"[[1, 2, 3], [4, 5, 6], … [10, 11, 12]]"
"[[1, 2, 3], [4, 5, 6], … [10, 11, 12]]"


### ⚠️ **שים לב**: טיפוסים מעורבים

אם יש ערכים מטיפוסים שונים, Polars ינסה להמיר הכל לטיפוס משותף:

In [13]:
# רשימה עם טיפוסים מעורבים
mixed_df = pl.DataFrame({
    'mixed_list': [
        [['a',2,3], [4,5,6], [7,8,9]], 
        [[1,2,3], [4,5,6], [7,8,9]]
    ]
}, strict=False)  # strict=False מאפשר המרות אוטומטיות

print("🔀 רשימה מעורבת:")
print(f"טיפוס: {mixed_df.dtypes[0]}")
mixed_df

🔀 רשימה מעורבת:
טיפוס: List(List(String))


mixed_list
list[list[str]]
"[[""a"", ""2"", ""3""], [""4"", ""5"", ""6""], [""7"", ""8"", ""9""]]"
"[[""1"", ""2"", ""3""], [""4"", ""5"", ""6""], [""7"", ""8"", ""9""]]"


---
<a id="צבירת-אלמנטים"></a>
## 🧮 2. צבירת אלמנטים ברשימות (Aggregating Elements)

### מהי צבירה?

**צבירה** (Aggregation) היא פעולה שלוקחת רשימה של ערכים ומחזירה **ערך בודד**.

### פעולות צבירה נפוצות:

| פעולה | תיאור | דוגמה |
|-------|--------|-------|
| `list.min()` | מינימום | `[3,1,4,1,5] → 1` |
| `list.max()` | מקסימום | `[3,1,4,1,5] → 5` |
| `list.mean()` | ממוצע | `[3,1,4,1,5] → 2.8` |
| `list.sum()` | סכום | `[3,1,4,1,5] → 14` |
| `list.len()` | אורך | `[3,1,4,1,5] → 5` |
| `list.join()` | איחוד למחרוזת | `['a','b','c'] → 'a:b:c'` |

### דוגמה מעשית: ניתוח סטטיסטי של צפיות

In [24]:
# קודם נצבור את הנתונים לרשימות
agg_df = (
    df
    .group_by('trending_date')
    .agg('views', 'likes', 'dislikes', 'comment_count')
    .sort('trending_date', descending=True)
)

print("📊 נתונים מצוברים:")
print(agg_df.shape)
agg_df.head()

📊 נתונים מצוברים:
(205, 5)


trending_date,views,likes,dislikes,comment_count
date,list[i64],list[i64],list[i64],list[i64]
2018-06-14,"[4427381, 5829270, … 10306119]","[96391, 87323, … 357079]","[5508, 3668, … 212976]","[12726, 11933, … 144795]"
2018-06-13,"[3238183, 470844, … 7839668]","[61841, 13922, … 352352]","[3708, 402, … 5871]","[0, 4843, … 46624]"
2018-06-12,"[3483553, 6173038, … 13619534]","[23725, 90478, … 347100]","[3145, 3877, … 6923]","[462, 7726, … 19977]"
2018-06-11,"[2341772, 846887, … 6995168]","[140374, 5758, … 143678]","[2951, 850, … 10925]","[33760, 1539, … 17444]"
2018-06-10,"[594004, 1504564, … 6980540]","[4470, 32430, … 143452]","[520, 4316, … 10911]","[1195, 6105, … 17429]"


In [26]:
# עכשיו נבצע פעולות צבירה על הרשימות
stats_df = agg_df.select(
    'trending_date',
    # סטטיסטיקות על צפיות
    pl.col('views').list.min().alias('min_views'),
    pl.col('views').list.max().alias('max_views'),
    pl.col('views').list.mean().alias('avg_views'),
    
    # סטטיסטיקות על לייקים
    pl.col('likes').list.max().alias('max_likes'),
    
    # סטטיסטיקות על דיסלייקים
    pl.col('dislikes').list.mean().alias('avg_dislikes'),
    
    # סכום תגובות
    pl.col('comment_count').list.sum().alias('total_comments')
).head()

print("📈 סטטיסטיקות מצפיות:")
for col in ['min_views', 'max_views', 'avg_views', 'max_likes', 'avg_dislikes', 'total_comments']:
    row = stats_df.sort(col, descending=True).row(0, named=True)
    print(f"{col}: {row[col]:,.0f} ({row['trending_date']})")
stats_df

📈 סטטיסטיקות מצפיות:
min_views: 189,038 (2018-06-14)
max_views: 94,254,507 (2018-06-14)
avg_views: 5,147,718 (2018-06-13)
max_likes: 2,032,463 (2018-06-14)
avg_dislikes: 8,541 (2018-06-13)
total_comments: 3,379,632 (2018-06-13)


trending_date,min_views,max_views,avg_views,max_likes,avg_dislikes,total_comments
date,i64,i64,f64,i64,f64,i64
2018-06-14,189038,94254507,5.1150e6,2032463,8472.94,3289433
2018-06-13,175754,89740876,5.1477e6,2021395,8541.23,3379632
2018-06-12,161782,85159766,4.9961e6,2004753,8353.685,3352328
2018-06-11,136643,80606384,4.997401e6,1981942,8478.095,3320709
2018-06-10,116841,78580312,4.9255e6,1967904,8405.22,3351105


### 💡 **פירוש התוצאות**

בתאריך 2018-06-14:
- **מינימום צפיות**: 189,038 (הסרטון הכי פחות נצפה)
- **מקסימום צפיות**: סרטון עם הכי הרבה צפיות
- **ממוצע צפיות**: הממוצע של כל הסרטונים
- **סה"ך תגובות**: סך כל התגובות על כל הסרטונים

### 🔹 איחוד רשימות למחרוזת (Join)

פעולת `join` מאחדת אלמנטים לתוך מחרוזת אחת:

In [ ]:
# איחוד שמות ערוצים
channel_summary = (
    df
    .group_by('trending_date')
    .agg(pl.col('channel_title'))
    .with_columns(
        pl.col('channel_title').list.join(', ').alias('all_channels')
    )
    .select('trending_date', 'all_channels')
    .sort('trending_date', descending=True)
).head(3)

print("📺 ערוצים לפי תאריך:")
channel_summary

📺 ערוצים לפי תאריך:


trending_date,all_channels
date,str
2018-06-14,"""Disney Movie Trailers, America…"
2018-06-13,"""Nintendo, GameXplain, gameslic…"
2018-06-12,"""gameslice, Clash of Clans, Fam…"


### 🔹 ספירת אלמנטים (Length)

כמה אלמנטים יש ברשימה?

In [89]:
# כמה סרטונים היו טרנדיים בכל יום?
video_counts = agg_df.select(
    'trending_date',
    pl.col('views').list.len().alias('num_trending_videos')
).head(10)

print("📊 מספר סרטונים טרנדיים ליום:")
video_counts

📊 מספר סרטונים טרנדיים ליום:


trending_date,num_trending_videos
date,u32
2018-06-14,200
2018-06-13,200
2018-06-12,200
2018-06-11,200
2018-06-10,200
2018-06-09,200
2018-06-08,200
2018-06-07,200
2018-06-06,200


### 🎓 תרגיל 2: צבירת נתונים

**משימה**: 
1. קבץ את הנתונים לפי `category_id`
2. מצא את מספר הצפיות הממוצע, המינימום והמקסימום לכל קטגוריה
3. מיין לפי ממוצע הצפיות (יורד)

In [117]:
res = (
    df
    .group_by('category_id')
    .agg([
        pl.col('views').mean().alias('avg_views'),
        pl.col('views').min().alias('min_views'),
        pl.col('views').max().alias('max_views')
    ])
    .head()
)

res

category_id,avg_views,min_views,max_views
i64,f64,i64,i64
26,983730.116739,1107,54155921
20,2.6208e6,1237,16935442
24,2.0679e6,798,149376127
2,1.3560e6,2860,25244097
23,1.4803e6,1807,29178096


---
<a id="גישה-ובחירה"></a>
## 🎯 3. גישה ובחירת אלמנטים (Accessing and Selecting Elements)

### איך ניגשים לאלמנטים ברשימה?

Polars מספק מגוון פעולות לבחירת אלמנטים ספציפיים מרשימות.

### טבלת פעולות גישה:

| פעולה | תיאור | דוגמה |
|-------|--------|-------|
| `list.first()` | האלמנט הראשון | `[1,2,3] → 1` |
| `list.last()` | האלמנט האחרון | `[1,2,3] → 3` |
| `list.get(n)` | אלמנט במיקום n | `[1,2,3].get(1) → 2` |
| `list.head(n)` | n אלמנטים ראשונים | `[1,2,3,4].head(2) → [1,2]` |
| `list.tail(n)` | n אלמנטים אחרונים | `[1,2,3,4].tail(2) → [3,4]` |
| `list.slice(offset, length)` | חיתוך מותאם | `[1,2,3,4].slice(1,2) → [2,3]` |

### הכנת הנתונים

In [118]:
# נצור DataFrame עם רשימות של תאריכים לכל ערוץ
trending_dates_by_channel = (
    df
    .group_by('channel_title')
    .agg('trending_date')
    .with_columns(
        # מיון התאריכים בתוך הרשימה
        pl.col('trending_date').list.sort()
    )
)

print("📺 תאריכים טרנדיים לפי ערוץ:")
trending_dates_by_channel.head()

📺 תאריכים טרנדיים לפי ערוץ:


channel_title,trending_date
str,list[date]
"""90s Commercials""",[2017-11-14]
"""郭韋辰""","[2017-12-07, 2017-12-08]"
"""MoreZoella""","[2017-12-09, 2017-12-10, … 2017-12-14]"
"""Physics Girl""","[2018-01-19, 2018-01-20, … 2018-06-14]"
"""Major League Soccer""","[2018-04-01, 2018-04-02, … 2018-04-19]"


### 🔹 אלמנטים ראשונים ואחרונים

In [119]:
# התאריך הראשון והאחרון שבו הערוץ היה טרנדי
first_last = trending_dates_by_channel.with_columns(
    pl.col('trending_date').list.first().alias('first_trending_date'),
    pl.col('trending_date').list.last().alias('last_trending_date')
).head()

print("📅 תאריכים ראשונים ואחרונים:")
first_last

📅 תאריכים ראשונים ואחרונים:


channel_title,trending_date,first_trending_date,last_trending_date
str,list[date],date,date
"""90s Commercials""",[2017-11-14],2017-11-14,2017-11-14
"""郭韋辰""","[2017-12-07, 2017-12-08]",2017-12-07,2017-12-08
"""MoreZoella""","[2017-12-09, 2017-12-10, … 2017-12-14]",2017-12-09,2017-12-14
"""Physics Girl""","[2018-01-19, 2018-01-20, … 2018-06-14]",2018-01-19,2018-06-14
"""Major League Soccer""","[2018-04-01, 2018-04-02, … 2018-04-19]",2018-04-01,2018-04-19


### 🔹 גישה לאלמנט במיקום ספציפי

In [134]:
# האלמנט ה-8 (אינדקס 7)
eighth_element = trending_dates_by_channel.with_columns(
    pl.col('trending_date').list.get(7, null_on_oob=True).alias('8th_trending_date')
).head()

print("🔢 האלמנט השמיני:")
eighth_element

🔢 האלמנט השמיני:


channel_title,trending_date,8th_trending_date
str,list[date],date
"""90s Commercials""",[2017-11-14],null
"""郭韋辰""","[2017-12-07, 2017-12-08]",null
"""MoreZoella""","[2017-12-09, 2017-12-10, … 2017-12-14]",null
"""Physics Girl""","[2018-01-19, 2018-01-20, … 2018-06-14]",2018-02-17
"""Major League Soccer""","[2018-04-01, 2018-04-02, … 2018-04-19]",2018-04-14


### 💡 **הסבר**: `null_on_oob=True`

- **OOB** = Out Of Bounds (מחוץ לגבולות)
- אם הרשימה קצרה מדי, החזר `null` במקום שגיאה
- בלי זה, תתקבל שגיאה אם הרשימה קצרה מ-8 אלמנטים

### 🔹 בחירת מספר אלמנטים

In [133]:
# 5 ראשונים ו-10 אחרונים
head_tail = trending_dates_by_channel.with_columns(
    pl.col('trending_date').list.head(5).alias('first_5_dates'),
    pl.col('trending_date').list.tail(10).alias('last_10_dates')
).head(3)

print("📊 ראשונים ואחרונים:")
head_tail

📊 ראשונים ואחרונים:


channel_title,trending_date,first_5_dates,last_10_dates
str,list[date],list[date],list[date]
"""90s Commercials""",[2017-11-14],[2017-11-14],[2017-11-14]
"""郭韋辰""","[2017-12-07, 2017-12-08]","[2017-12-07, 2017-12-08]","[2017-12-07, 2017-12-08]"
"""MoreZoella""","[2017-12-09, 2017-12-10, … 2017-12-14]","[2017-12-09, 2017-12-10, … 2017-12-13]","[2017-12-09, 2017-12-10, … 2017-12-14]"


### 🔹 חיתוך מתקדם (Slicing)

חיתוך דומה ל-Python slicing, אבל עם תחביר שונה:

In [135]:
# דוגמאות לחיתוך
slicing_examples = trending_dates_by_channel.select(
    'trending_date',
    # 2 ראשונים (התחל מ-0, קח 2)
    pl.col('trending_date').list.slice(0, 2).alias('first_2'),
    
    # האלמנט השלישי מהסוף (התחל מ--3, קח 1)
    pl.col('trending_date').list.slice(-3, 1).alias('3rd_from_end'),
    
    # מהאלמנט ה-8 עד הסוף
    pl.col('trending_date').list.slice(7).alias('from_8th_onward')
).head()

print("✂️ דוגמאות חיתוך:")
slicing_examples

✂️ דוגמאות חיתוך:


trending_date,first_2,3rd_from_end,from_8th_onward
list[date],list[date],list[date],list[date]
[2017-11-14],[2017-11-14],[],[]
"[2017-12-07, 2017-12-08]","[2017-12-07, 2017-12-08]",[],[]
"[2017-12-09, 2017-12-10, … 2017-12-14]","[2017-12-09, 2017-12-10]",[2017-12-12],[]
"[2018-01-19, 2018-01-20, … 2018-06-14]","[2018-01-19, 2018-01-20]",[2018-06-12],"[2018-02-17, 2018-02-18, … 2018-06-14]"
"[2018-04-01, 2018-04-02, … 2018-04-19]","[2018-04-01, 2018-04-02]",[2018-04-17],"[2018-04-14, 2018-04-15, … 2018-04-19]"


### 💡 **תחביר slice**

```python
list.slice(offset, length)
```

- **offset**: נקודת ההתחלה (יכול להיות שלילי לספירה מהסוף)
- **length**: כמה אלמנטים לקחת (אופציונלי, ברירת מחדל עד הסוף)

### 🔹 בחירת אלמנטים מרובים (Gather)

`gather` מאפשר לבחור אלמנטים במיקומים לא רציפים:

In [136]:
# בחירת אלמנטים ספציפיים
gathered = trending_dates_by_channel.select(
    'trending_date',
    # ראשון ואחרון
    pl.col('trending_date').list.gather([0, -1]).alias('first_and_last'),
    
    # ראשון 4 פעמים
    pl.col('trending_date').list.gather([0, 0, 0, 0]).alias('first_repeated'),
    
    # ראשון ו-11 (עם null אם לא קיים)
    pl.col('trending_date').list.gather([0, 10], null_on_oob=True).alias('first_and_11th')
).head()

print("🎯 בחירה מותאמת:")
gathered

🎯 בחירה מותאמת:


trending_date,first_and_last,first_repeated,first_and_11th
list[date],list[date],list[date],list[date]
[2017-11-14],"[2017-11-14, 2017-11-14]","[2017-11-14, 2017-11-14, … 2017-11-14]","[2017-11-14, null]"
"[2017-12-07, 2017-12-08]","[2017-12-07, 2017-12-08]","[2017-12-07, 2017-12-07, … 2017-12-07]","[2017-12-07, null]"
"[2017-12-09, 2017-12-10, … 2017-12-14]","[2017-12-09, 2017-12-14]","[2017-12-09, 2017-12-09, … 2017-12-09]","[2017-12-09, null]"
"[2018-01-19, 2018-01-20, … 2018-06-14]","[2018-01-19, 2018-06-14]","[2018-01-19, 2018-01-19, … 2018-01-19]","[2018-01-19, 2018-02-20]"
"[2018-04-01, 2018-04-02, … 2018-04-19]","[2018-04-01, 2018-04-19]","[2018-04-01, 2018-04-01, … 2018-04-01]","[2018-04-01, 2018-04-17]"


### 🎓 תרגיל 3: גישה לאלמנטים

**משימה**: 
1. צור DataFrame שמקבץ את כל הכותרות לפי קטגוריה
2. הצג את 3 הכותרות הראשונות מכל קטגוריה
3. הצג את הכותרת האמצעית (באמצע הרשימה)

In [ ]:
# כתוב את הקוד שלך כאן:
# רמז: list.get(pl.col('title').list.len() // 2)



### 🔹 פעולות נוספות: מיון ודגימה

In [137]:
# 3 התאריכים האחרונים (לאחר מיון)
recent_dates = trending_dates_by_channel.with_columns(
    pl.col('trending_date')
    .list.sort(descending=True)  # מיין בסדר יורד
    .list.head(3)  # קח 3 ראשונים
    .alias('3_most_recent')
).head()

print("📅 3 התאריכים האחרונים:")
recent_dates

📅 3 התאריכים האחרונים:


channel_title,trending_date,3_most_recent
str,list[date],list[date]
"""90s Commercials""",[2017-11-14],[2017-11-14]
"""郭韋辰""","[2017-12-07, 2017-12-08]","[2017-12-08, 2017-12-07]"
"""MoreZoella""","[2017-12-09, 2017-12-10, … 2017-12-14]","[2017-12-14, 2017-12-13, 2017-12-12]"
"""Physics Girl""","[2018-01-19, 2018-01-20, … 2018-06-14]","[2018-06-14, 2018-06-13, 2018-06-12]"
"""Major League Soccer""","[2018-04-01, 2018-04-02, … 2018-04-19]","[2018-04-19, 2018-04-18, 2018-04-17]"


---
<a id="החלת-לוגיקה"></a>
## ⚙️ 4. החלת לוגיקה על אלמנטים (Applying Logic to Elements)

### מהי `list.eval()`?

`list.eval()` מאפשר להחיל **פעולה על כל אלמנט** ברשימה בנפרד.

### תחביר בסיסי:

```python
pl.col('my_list').list.eval(
    פעולה_על_כל_אלמנט
)
```

### הקשרים מיוחדים:

- `pl.element()` - האלמנט הנוכחי
- `pl.first()` - האלמנט הראשון ברשימה
- `pl.last()` - האלמנט האחרון ברשימה
- `pl.col('')` - כל העמודה (השורה הנוכחית)

### הכנת נתונים

In [138]:
# נצור DataFrame עם רשימות של צפיות וערוצים
agg_df = (
    df
    .group_by('trending_date')
    .agg('views', 'channel_title')
)

print("📊 נתונים מצוברים:")
agg_df.head()

📊 נתונים מצוברים:


trending_date,views,channel_title
date,list[i64],list[str]
2018-02-04,"[209539, 1497616, … 180238]","[""This Is Us"", ""FBE"", … ""Sports Illustrated""]"
2018-03-03,"[716096, 1108125, … 1327815]","[""carrieunderwoodVEVO"", ""nigahiga"", … ""Brian Hull""]"
2018-02-07,"[10582444, 1894717, … 39595]","[""SpaceX"", ""KendrickLamarVEVO"", … ""Us Weekly""]"
2018-02-10,"[2768013, 1355968, … 2666775]","[""CaseyNeistat"", ""Butch Hartman"", … ""MLive""]"
2018-02-16,"[10096037, 744691, … 302660]","[""Disney•Pixar"", ""Mr. Kate"", … ""ColdFusion""]"


### 🔹 החזרת הרשימה כמות שהיא

בואו נראה את ההבדל בין הקשרים שונים:

In [139]:
# כל ההקשרים מחזירים את אותה רשימה (כי אין פעולה)
contexts = agg_df.select(
    'views',
    pl.col('views').list.eval(pl.element()).alias('element_context'),
    pl.col('views').list.eval(pl.first()).alias('first_context'),
    pl.col('views').list.eval(pl.last()).alias('last_context'),
    pl.col('views').list.eval(pl.col('')).alias('col_context')
).head(3)

print("🔍 הקשרים שונים:")
contexts

🔍 הקשרים שונים:


views,element_context,first_context,last_context,col_context
list[i64],list[i64],list[i64],list[i64],list[i64]
"[209539, 1497616, … 180238]","[209539, 1497616, … 180238]","[209539, 1497616, … 180238]","[209539, 1497616, … 180238]","[209539, 1497616, … 180238]"
"[716096, 1108125, … 1327815]","[716096, 1108125, … 1327815]","[716096, 1108125, … 1327815]","[716096, 1108125, … 1327815]","[716096, 1108125, … 1327815]"
"[10582444, 1894717, … 39595]","[10582444, 1894717, … 39595]","[10582444, 1894717, … 39595]","[10582444, 1894717, … 39595]","[10582444, 1894717, … 39595]"


### 🔹 המרה לאותיות גדולות

In [140]:
# המרת שמות ערוצים לאותיות גדולות
uppercase_channels = (
    agg_df
    .select(
        pl.col('channel_title').list.head(2),
        pl.col('channel_title')
        .list.eval(pl.element().str.to_uppercase())
        .list.head(2)
        .alias('channel_uppercase')
    )
).head()

print("🔤 המרה לאותיות גדולות:")
uppercase_channels

🔤 המרה לאותיות גדולות:


channel_title,channel_uppercase
list[str],list[str]
"[""This Is Us"", ""FBE""]","[""THIS IS US"", ""FBE""]"
"[""carrieunderwoodVEVO"", ""nigahiga""]","[""CARRIEUNDERWOODVEVO"", ""NIGAHIGA""]"
"[""SpaceX"", ""KendrickLamarVEVO""]","[""SPACEX"", ""KENDRICKLAMARVEVO""]"
"[""CaseyNeistat"", ""Butch Hartman""]","[""CASEYNEISTAT"", ""BUTCH HARTMAN""]"
"[""Disney•Pixar"", ""Mr. Kate""]","[""DISNEY•PIXAR"", ""MR. KATE""]"


### 💡 **הסבר מפורט**

1. `pl.col('channel_title')` - בוחר את עמודת הערוצים
2. `.list.eval(...)` - החל פעולה על כל אלמנט ברשימה
3. `pl.element()` - האלמנט הנוכחי (שם ערוץ)
4. `.str.to_uppercase()` - המר לאותיות גדולות
5. `.list.head(2)` - קח רק 2 ראשונים (להצגה)

### 🔹 סינון אלמנטים

נוכל לסנן אלמנטים שמקיימים תנאי:

In [141]:
# השאר רק ערוצים שמכילים את האות A
filtered_channels = uppercase_channels.with_columns(
    pl.col('channel_uppercase')
    .list.eval(
        pl.element().filter(
            pl.element().str.contains('A', literal=True)
        )
    )
).head()

print("🔍 ערוצים עם A:")
filtered_channels

🔍 ערוצים עם A:


channel_title,channel_uppercase
list[str],list[str]
"[""This Is Us"", ""FBE""]",[]
"[""carrieunderwoodVEVO"", ""nigahiga""]","[""CARRIEUNDERWOODVEVO"", ""NIGAHIGA""]"
"[""SpaceX"", ""KendrickLamarVEVO""]","[""SPACEX"", ""KENDRICKLAMARVEVO""]"
"[""CaseyNeistat"", ""Butch Hartman""]","[""CASEYNEISTAT"", ""BUTCH HARTMAN""]"
"[""Disney•Pixar"", ""Mr. Kate""]","[""DISNEY•PIXAR"", ""MR. KATE""]"


### 🔹 חישוב דירוג (Ranking)

In [142]:
# דירוג צפיות בתוך כל יום
ranked_views = agg_df.select(
    'trending_date',
    'views',
    pl.col('views')
    .list.eval(
        pl.element().rank('dense', descending=True)
    )
    .alias('views_rank')
).head()

print("🏆 דירוג צפיות:")
ranked_views

🏆 דירוג צפיות:


trending_date,views,views_rank
date,list[i64],list[u32]
2018-02-04,"[209539, 1497616, … 180238]","[122, 29, … 131]"
2018-03-03,"[716096, 1108125, … 1327815]","[80, 55, … 47]"
2018-02-07,"[10582444, 1894717, … 39595]","[10, 31, … 185]"
2018-02-10,"[2768013, 1355968, … 2666775]","[23, 45, … 24]"
2018-02-16,"[10096037, 744691, … 302660]","[7, 73, … 113]"


### 💡 **הסבר**: מה זה דירוג?

- המספר **1** = הכי הרבה צפיות
- המספר **2** = שני בגודל
- וכך הלאה...

לדוגמה: `[748374, 2418783, 3191434]` → `[3, 2, 1]`

### 🔹 חישובים מורכבים: ההפרש מהמקסימום

In [143]:
# כמה צפיות חסרות לכל סרטון כדי להיות הכי גבוה?
diff_from_max = ranked_views.select(
    'views',
    pl.col('views')
    .list.eval(
        pl.element().max() - pl.element()
    )
    .alias('gap_to_max')
).head()

print("📊 פער מהמקסימום:")
diff_from_max

📊 פער מהמקסימום:


views,gap_to_max
list[i64],list[i64]
"[209539, 1497616, … 180238]","[21384800, 20096723, … 21414101]"
"[716096, 1108125, … 1327815]","[16779621, 16387592, … 16167902]"
"[10582444, 1894717, … 39595]","[30218979, 38906706, … 40761828]"
"[2768013, 1355968, … 2666775]","[48475136, 49887181, … 48576374]"
"[10096037, 744691, … 302660]","[10562603, 19913949, … 20355980]"


### 🔹 פעולות על קבוצות (Set Operations)

In [144]:
# נשווה את הדירוגים של 2 הראשונים מול 2 האחרונים
set_operations = ranked_views.with_columns(
    pl.col('views_rank').list.slice(0, 2).alias('top_2_ranks'),
    pl.col('views_rank').list.slice(-2, 2).alias('bottom_2_ranks')
).select(
    'top_2_ranks',
    'bottom_2_ranks',
    # חיתוך (מה משותף)
    pl.col('top_2_ranks').list.set_intersection('bottom_2_ranks').alias('intersection'),
    # איחוד (כל הערכים הייחודיים)
    pl.col('top_2_ranks').list.set_union('bottom_2_ranks').alias('union'),
    # הפרש (מה ב-A אבל לא ב-B)
    pl.col('top_2_ranks').list.set_difference('bottom_2_ranks').alias('difference')
).head()

print("🔀 פעולות קבוצות:")
set_operations

🔀 פעולות קבוצות:


top_2_ranks,bottom_2_ranks,intersection,union,difference
list[u32],list[u32],list[u32],list[u32],list[u32]
"[122, 29]","[146, 131]",[],"[122, 29, … 131]","[122, 29]"
"[80, 55]","[64, 47]",[],"[80, 55, … 47]","[80, 55]"
"[10, 31]","[34, 185]",[],"[10, 31, … 185]","[10, 31]"
"[23, 45]","[74, 24]",[],"[23, 45, … 24]","[23, 45]"
"[7, 73]","[136, 113]",[],"[7, 73, … 113]","[7, 73]"


### 🎓 תרגיל 4: החלת לוגיקה

**משימה**:
1. צור רשימה של מספרי הלייקים לפי תאריך
2. חשב עבור כל סרטון: `likes / (likes + dislikes)` - אחוז הלייקים
3. מצא את הממוצע של אחוז הלייקים בכל יום

In [ ]:
# כתוב את הקוד שלך כאן:



### 🔹 פיצול רשימות (Explode)

לפעמים נרצה "לפרוס" רשימה - להפוך כל אלמנט לשורה נפרדת:

In [145]:
# דוגמה: פריסת רשימת הצפיות
exploded_example = (
    ranked_views
    .head(2)  # קח רק 2 תאריכים
    .select('trending_date', 'views', 'views_rank')
    .explode('views', 'views_rank')  # פרוס את הרשימות
)

print("💥 לפני explode:")
print(ranked_views.head(2))
print("\n💥 אחרי explode:")
print(exploded_example)

💥 לפני explode:
shape: (2, 3)
┌───────────────┬──────────────────────────────┬──────────────────┐
│ trending_date ┆ views                        ┆ views_rank       │
│ ---           ┆ ---                          ┆ ---              │
│ date          ┆ list[i64]                    ┆ list[u32]        │
╞═══════════════╪══════════════════════════════╪══════════════════╡
│ 2018-02-04    ┆ [209539, 1497616, … 180238]  ┆ [122, 29, … 131] │
│ 2018-03-03    ┆ [716096, 1108125, … 1327815] ┆ [80, 55, … 47]   │
└───────────────┴──────────────────────────────┴──────────────────┘

💥 אחרי explode:
shape: (395, 3)
┌───────────────┬─────────┬────────────┐
│ trending_date ┆ views   ┆ views_rank │
│ ---           ┆ ---     ┆ ---        │
│ date          ┆ i64     ┆ u32        │
╞═══════════════╪═════════╪════════════╡
│ 2018-02-04    ┆ 209539  ┆ 122        │
│ 2018-02-04    ┆ 1497616 ┆ 29         │
│ 2018-02-04    ┆ 39720   ┆ 174        │
│ 2018-02-04    ┆ 2418179 ┆ 19         │
│ 2018-02-04    ┆ 471957

---
<a id="עבודה-עם-structs"></a>
## 🏗️ 5. עבודה עם Structs ו-JSON

### מהו Struct?

**Struct** הוא מבנה נתונים שמכיל מספר **שדות** (fields), כמו dictionary ב-Python.

**דוגמה**:
```python
{
    'name': 'John',
    'age': 30,
    'city': 'Tel Aviv'
}
```

### למה להשתמש ב-Structs?

✅ **ארגון טוב יותר**: מידע קשור נשמר ביחד  
✅ **יעילות**: פחות עמודות, יותר מבנה  
✅ **עבודה עם JSON**: המרה קלה מ/אל JSON

### טעינת נתוני JSON

In [150]:
# טעינת נתוני Google Analytics בפורמט JSON
# נתונים אלו מכילים structs מקוננים

ga_df = pl.read_json('../data/ga_20170801.json', infer_schema_length=10000)

# נבחר כמה עמודות רלוונטיות
cols = ['visitId', 'date', 'totals', 'trafficSource', 'customDimensions', 'channelGrouping']
ga_df = ga_df.select(cols)

print(f"📊 מספר ביקורים: {ga_df.height:,}")
print(f"\n🔍 5 ביקורים ראשונים:")
ga_df.head()

📊 מספר ביקורים: 1,000

🔍 5 ביקורים ראשונים:


visitId,date,totals,trafficSource,customDimensions,channelGrouping
str,str,struct[13],struct[9],list[struct[2]],str
"""1501591568""","""20170801""","{""1"",""1"",""1"",null,""1"",null,null,""1"",null,null,null,null,""1""}","{null,""(not set)"",""(direct)"",""(none)"",null,null,{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null},null,null}",[],"""Organic Search"""
"""1501589647""","""20170801""","{""1"",""1"",""1"",null,""1"",null,null,null,null,null,null,null,""1""}","{""/analytics/web/"",""(not set)"",""analytics.google.com"",""referral"",null,null,{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null},null,null}","[{""4"",""APAC""}]","""Referral"""
"""1501616621""","""20170801""","{""1"",""1"",""1"",null,""1"",null,null,""1"",null,null,null,null,""1""}","{""/analytics/web/"",""(not set)"",""analytics.google.com"",""referral"",null,null,{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null},null,null}","[{""4"",""EMEA""}]","""Referral"""
"""1501601200""","""20170801""","{""1"",""1"",""1"",null,""1"",null,null,""1"",null,null,null,null,""1""}","{""/analytics/web/"",""(not set)"",""analytics.google.com"",""referral"",null,null,{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null},null,null}","[{""4"",""North America""}]","""Referral"""
"""1501615525""","""20170801""","{""1"",""1"",""1"",null,""1"",null,null,""1"",null,null,null,null,""1""}","{""/analytics/web/"",""(not set)"",""adwords.google.com"",""referral"",null,null,{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null},null,null}","[{""4"",""North America""}]","""Referral"""


### בחינת הטיפוסים

In [151]:
# בואו נראה את טיפוסי העמודות
print("📋 טיפוסי העמודות:")
for col in ga_df.columns:
    dtype = ga_df[col].dtype
    print(f"  {col}: {dtype}")

📋 טיפוסי העמודות:
  visitId: String
  date: String
  totals: Struct({'visits': String, 'hits': String, 'pageviews': String, 'timeOnSite': Null, 'bounces': String, 'transactions': Null, 'transactionRevenue': Null, 'newVisits': String, 'screenviews': Null, 'uniqueScreenviews': Null, 'timeOnScreen': Null, 'totalTransactionRevenue': Null, 'sessionQualityDim': String})
  trafficSource: Struct({'referralPath': String, 'campaign': String, 'source': String, 'medium': String, 'keyword': String, 'adContent': String, 'adwordsClickInfo': Struct({'campaignId': Null, 'adGroupId': Null, 'creativeId': Null, 'criteriaId': Null, 'page': String, 'slot': String, 'criteriaParameters': String, 'gclId': String, 'customerId': Null, 'adNetworkType': String, 'targetingCriteria': Null, 'isVideoAd': String}), 'isTrueDirect': String, 'campaignCode': Null})
  customDimensions: List(Struct({'index': String, 'value': String}))
  channelGrouping: String


### 🔹 יצירת Struct מעמודות קיימות

In [152]:
# נצור struct מ-3 עמודות
with_struct = ga_df.with_columns(
    pl.struct('visitId', 'date', 'channelGrouping').alias('visit_info')
)

print("🏗️ Struct שנוצר:")
with_struct.select('visitId', 'date', 'channelGrouping', 'visit_info').head()

🏗️ Struct שנוצר:


visitId,date,channelGrouping,visit_info
str,str,str,struct[3]
"""1501591568""","""20170801""","""Organic Search""","{""1501591568"",""20170801"",""Organic Search""}"
"""1501589647""","""20170801""","""Referral""","{""1501589647"",""20170801"",""Referral""}"
"""1501616621""","""20170801""","""Referral""","{""1501616621"",""20170801"",""Referral""}"
"""1501601200""","""20170801""","""Referral""","{""1501601200"",""20170801"",""Referral""}"
"""1501615525""","""20170801""","""Referral""","{""1501615525"",""20170801"",""Referral""}"


### 💡 **מבנה ה-Struct**

העמודה `visit_info` מכילה struct עם 3 שדות:
- `visitId`
- `date`
- `channelGrouping`

כל תא ב-`visit_info` הוא מבנה שמכיל את 3 הערכים!

### 🔹 פתיחת Struct (Unnesting)

In [153]:
# נפרק את ה-struct חזרה לעמודות
unnested = (
    with_struct
    .select('visit_info')
    .unnest('visit_info')
).head()

print("📤 Struct שנפרק:")
unnested

📤 Struct שנפרק:


visitId,date,channelGrouping
str,str,str
"""1501591568""","""20170801""","""Organic Search"""
"""1501589647""","""20170801""","""Referral"""
"""1501616621""","""20170801""","""Referral"""
"""1501601200""","""20170801""","""Referral"""
"""1501615525""","""20170801""","""Referral"""


### 🔹 גישה לשדה ספציפי ב-Struct

In [154]:
# גישה לשדה ספציפי
field_access = with_struct.select(
    'visit_info',
    pl.col('visit_info').struct.field('channelGrouping').alias('channel')
).head()

print("🎯 גישה לשדה:")
field_access

🎯 גישה לשדה:


visit_info,channel
struct[3],str
"{""1501591568"",""20170801"",""Organic Search""}","""Organic Search"""
"{""1501589647"",""20170801"",""Referral""}","""Referral"""
"{""1501616621"",""20170801"",""Referral""}","""Referral"""
"{""1501601200"",""20170801"",""Referral""}","""Referral"""
"{""1501615525"",""20170801"",""Referral""}","""Referral"""


### 🔹 שינוי שמות שדות

In [155]:
# שינוי שמות השדות ב-struct
renamed = with_struct.select(
    'visit_info',
    pl.col('visit_info')
    .struct.rename_fields(['id', 'date_str', 'channel'])
    .alias('visit_info_renamed')
).head()

print("✏️ שמות שדות חדשים:")
renamed

✏️ שמות שדות חדשים:


visit_info,visit_info_renamed
struct[3],struct[3]
"{""1501591568"",""20170801"",""Organic Search""}","{""1501591568"",""20170801"",""Organic Search""}"
"{""1501589647"",""20170801"",""Referral""}","{""1501589647"",""20170801"",""Referral""}"
"{""1501616621"",""20170801"",""Referral""}","{""1501616621"",""20170801"",""Referral""}"
"{""1501601200"",""20170801"",""Referral""}","{""1501601200"",""20170801"",""Referral""}"
"{""1501615525"",""20170801"",""Referral""}","{""1501615525"",""20170801"",""Referral""}"


### 🔹 עבודה עם Struct קיים

In [156]:
# נפרק את ה-struct של trafficSource
traffic_unnested = (
    ga_df
    .select('trafficSource')
    .unnest('trafficSource')
).head()

print("🚦 מקור תעבורה מפורק:")
traffic_unnested

🚦 מקור תעבורה מפורק:


referralPath,campaign,source,medium,keyword,adContent,adwordsClickInfo,isTrueDirect,campaignCode
str,str,str,str,str,str,struct[12],str,null
null,"""(not set)""","""(direct)""","""(none)""",null,null,"{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null}",null,null
"""/analytics/web/""","""(not set)""","""analytics.google.com""","""referral""",null,null,"{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null}",null,null
"""/analytics/web/""","""(not set)""","""analytics.google.com""","""referral""",null,null,"{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null}",null,null
"""/analytics/web/""","""(not set)""","""analytics.google.com""","""referral""",null,null,"{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null}",null,null
"""/analytics/web/""","""(not set)""","""adwords.google.com""","""referral""",null,null,"{null,null,null,null,null,null,""not available in demo dataset"",null,null,null,null,null}",null,null


### 🔹 שילוב Structs וגישה לשדות

In [157]:
# נשלב ערוץ עם מקור התעבורה
combined = ga_df.select(
    pl.struct(
        pl.col('channelGrouping'),
        pl.col('trafficSource').struct.field('source')
    )
    .unique()
    .alias('channel_and_source')
).unnest('channel_and_source').sort('channelGrouping', 'source')

print("🔀 שילוב ערוץ ומקור:")
combined

🔀 שילוב ערוץ ומקור:


channelGrouping,source
str,str
"""Affiliates""","""Partners"""
"""Direct""","""(direct)"""
"""Display""","""(direct)"""
"""Display""","""dfa"""
"""Organic Search""","""(direct)"""
…,…
"""Social""","""groups.google.com"""
"""Social""","""l.facebook.com"""
"""Social""","""m.facebook.com"""


### 🔹 המרה ל-JSON וחזרה

In [159]:
# המרת struct ל-JSON
json_conversion = (
    ga_df
    .select(
        # המרה ל-JSON string
        pl.col('totals').struct.json_encode().alias('totals_json'),
        # המרה חזרה ל-struct
        pl.col('totals').struct.json_encode().str.json_decode(dtype=pl.Struct).alias('totals_decoded')

    )
).head()

print("🔄 המרה ל-JSON וחזרה:")
json_conversion

🔄 המרה ל-JSON וחזרה:


totals_json,totals_decoded
str,struct[0]
"""{""visits"":""1"",""hits"":""1"",""page…",{}
"""{""visits"":""1"",""hits"":""1"",""page…",{}
"""{""visits"":""1"",""hits"":""1"",""page…",{}
"""{""visits"":""1"",""hits"":""1"",""page…",{}
"""{""visits"":""1"",""hits"":""1"",""page…",{}


### 💡 **מתי להשתמש ב-JSON encoding/decoding?**

- ✅ שמירת נתונים לקובץ
- ✅ העברת נתונים בין מערכות
- ✅ אחסון במסד נתונים
- ✅ API requests/responses

### 🔹 המרת רשימה ל-Struct

In [168]:
# נקבץ נתונים ונהפוך רשימה ל-struct
list_to_struct = (
    ga_df
    .group_by('channelGrouping')
    .agg([
        pl.col('visitId'),
        pl.col('visitId').len().alias('numVisits')
        ])
    .sort('numVisits')
    .with_columns(
        # המרת הרשימה ל-struct (כל אלמנט הופך לשדה)
        pl.col('visitId').list.to_struct(upper_bound=10).alias('visits_struct')
    )
)

print("📊 רשימה שהומרה ל-struct:")
list_to_struct

📊 רשימה שהומרה ל-struct:


channelGrouping,visitId,numVisits,visits_struct
str,list[str],u32,struct[10]
"""Display""","[""1501651856"", ""1501625928"", … ""1501638116""]",12,"{""1501651856"",""1501625928"",""1501611633"",""1501625068"",""1501612878"",""1501616158"",""1501607332"",""1501622703"",""1501621231"",""1501649570""}"
"""Paid Search""","[""1501610896"", ""1501644116"", … ""1501613648""]",20,"{""1501610896"",""1501644116"",""1501574187"",""1501614708"",""1501625398"",""1501636143"",""1501617844"",""1501628199"",""1501618328"",""1501624893""}"
"""Affiliates""","[""1501604627"", ""1501572101"", … ""1501635918""]",29,"{""1501604627"",""1501572101"",""1501638418"",""1501589595"",""1501635523"",""1501588687"",""1501602677"",""1501656633"",""1501589444"",""1501654572""}"
"""Referral""","[""1501589647"", ""1501616621"", … ""1501607798""]",106,"{""1501589647"",""1501616621"",""1501601200"",""1501615525"",""1501589650"",""1501573710"",""1501613382"",""1501630140"",""1501656976"",""1501602227""}"
"""Social""","[""1501590147"", ""1501655923"", … ""1501652602""]",136,"{""1501590147"",""1501655923"",""1501640054"",""1501596419"",""1501591307"",""1501616949"",""1501649584"",""1501579329"",""1501585058"",""1501618027""}"
"""Direct""","[""1501586309"", ""1501587435"", … ""1501610792""]",163,"{""1501586309"",""1501587435"",""1501653660"",""1501608816"",""1501611913"",""1501584277"",""1501578373"",""1501587465"",""1501621325"",""1501655032""}"
"""Organic Search""","[""1501591568"", ""1501583103"", … ""1501625964""]",534,"{""1501591568"",""1501583103"",""1501631547"",""1501599064"",""1501585229"",""1501639903"",""1501576309"",""1501573981"",""1501618526"",""1501578968""}"


### 🎓 תרגיל 5: עבודה עם Structs

**משימה**:
1. צור struct שמכיל: `video_id`, `title`, `views`, `likes`
2. המר אותו ל-JSON
3. פרק את ה-struct והצג רק את `title` ו-`views`

In [ ]:
# כתוב את הקוד שלך כאן:



---
<a id="תרגילים"></a>
## 🎯 תרגילים ואתגרים נוספים

### תרגיל 6: ניתוח מתקדם של תגיות

**משימה מורכבת**:
1. פצל את עמודת `tags` לרשימה
2. מצא את 10 התגיות הכי נפוצות
3. עבור כל תגית, חשב את ממוצע הצפיות של סרטונים עם התגית הזו
4. הצג גרף (אופציונלי)

In [ ]:
# כתוב את הקוד שלך כאן:
# רמזים:
# 1. str.split('|')
# 2. explode()
# 3. group_by() + agg()
# 4. value_counts()



### תרגיל 7: מציאת סרטונים "ויראליים"

**משימה**:
1. עבור כל ערוץ, צור רשימה של מספרי הצפיות
2. חשב את הסטיית התקן של הצפיות
3. מצא סרטונים שהצפיות שלהם גבוהות פי 3+ מהממוצע
4. הצג את הערוצים עם הכי הרבה סרטונים "ויראליים"

In [ ]:
# כתוב את הקוד שלך כאן:



### תרגיל 8: ניתוח זמני (Temporal Analysis)

**משימה**:
1. לכל ערוץ, צור רשימה של התאריכים שבהם היה טרנדי
2. חשב את ההפרשים בין תאריכים עוקבים
3. מצא ערוצים שהיו טרנדיים באופן עקבי (פער קטן בין תאריכים)

In [ ]:
# כתוב את הקוד שלך כאן:



---
<a id="סיכום"></a>
## 📚 סיכום ומשאבים נוספים

### סיכום המושגים העיקריים

| נושא | פעולות עיקריות | דוגמה |
|------|----------------|-------|
| **יצירת רשימות** | `str.split()`, `group_by().agg()`, `concat_list()` | `pl.col('tags').str.split('│')` |
| **צבירה** | `list.min/max/mean/sum/len()` | `pl.col('views').list.mean()` |
| **גישה לאלמנטים** | `list.first/last/get/head/tail()` | `pl.col('dates').list.first()` |
| **חיתוך** | `list.slice()`, `list.gather()` | `pl.col('items').list.slice(0,3)` |
| **החלת לוגיקה** | `list.eval()` | `pl.col('lst').list.eval(pl.element()*2)` |
| **Structs** | `struct()`, `unnest()`, `struct.field()` | `pl.struct('a','b','c')` |
| **JSON** | `struct.json_encode/decode()` | `pl.col('data').struct.json_encode()` |

### טיפים וטריקים חשובים

#### 1. ⚡ ביצועים
```python
# ✅ טוב - פעולה אחת
df.with_columns(
    pl.col('tags').str.split('|').alias('tags_list')
)

# ❌ פחות טוב - פעולות מרובות
df.with_columns(
    pl.col('tags').str.split('|')
).with_columns(
    pl.col('tags').list.len()
)
```

#### 2. 🛡️ טיפול ב-nulls
```python
# תמיד השתמש ב-null_on_oob
pl.col('my_list').list.get(10, null_on_oob=True)
```

#### 3. 🔍 דיבאג
```python
# השתמש ב-glimpse() לבדוק טיפוסים
df.glimpse()

# הצג מספר מוגבל של אלמנטים
pl.col('my_list').list.head(3)
```

#### 4. 🎨 קריאות קוד
```python
# ✅ קריא - שלבים ברורים
(
    df
    .group_by('date')
    .agg('views')
    .with_columns(
        pl.col('views').list.mean().alias('avg_views')
    )
)
```

### מלכודות נפוצות (Common Pitfalls)

#### ❌ לא להשתמש ב-eval בלי צורך
```python
# ❌ מיותר
pl.col('views').list.eval(pl.element().sum())

# ✅ פשוט יותר
pl.col('views').list.sum()
```

#### ❌ לשכוח null_on_oob
```python
# ❌ עלול לזרוק שגיאה
pl.col('items').list.get(100)

# ✅ בטוח
pl.col('items').list.get(100, null_on_oob=True)
```

#### ❌ ערבוב explode עם agg
```python
# ❌ לא עובד
df.explode('tags').group_by('date').agg('tags')

# ✅ נכון
df.group_by('date').agg('tags')
```

### משאבים נוספים

#### 📖 תיעוד רשמי
- [Polars User Guide](https://pola-rs.github.io/polars/user-guide/)
- [API Reference](https://pola-rs.github.io/polars/py-polars/html/reference/)
- [List Functions](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/list.html)

#### 🎓 קורסים ומדריכים
- [Polars Tutorial on Real Python](https://realpython.com/polars-python/)
- [Modern Polars (GitHub)](https://github.com/pola-rs/polars-book)

#### 💬 קהילה
- [Polars Discord Server](https://discord.gg/4UfP5cfBE7)
- [Stack Overflow - polars tag](https://stackoverflow.com/questions/tagged/polars)
- [GitHub Discussions](https://github.com/pola-rs/polars/discussions)

#### 🎬 וידאו
- [Polars YouTube Channel](https://www.youtube.com/@polarsDataFrame)

---

### 🎉 סיימתם!

**מזל טוב!** עכשיו אתם יודעים:
- ✅ ליצור ולנהל רשימות ב-Polars
- ✅ לצבור ולנתח נתונים מקוננים
- ✅ לגשת ולבחור אלמנטים ברשימות
- ✅ להחיל פעולות מורכבות על רשימות
- ✅ לעבוד עם Structs ו-JSON

**הצעד הבא**: נסו להשתמש בטכניקות אלו בפרויקט אמיתי שלכם! 🚀

---

### 📝 משוב

מצאתם שגיאה? יש הצעות לשיפור? אנא פנו אלינו!

**בהצלחה בדרך שלכם עם Polars!** 🐻‍❄️